In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import get_agnews_prepraped
from models import SentenceCNN

In [2]:
device = 'cpu'
size_train_batch = 64
size_test_batch = 1028

train_set, test_set, vocab_size, n_classes, vocab = get_agnews_prepraped(random_state=42,
                                                                   batch_sizes=(size_train_batch, size_test_batch))
X_test, Y_test = next(iter(test_set))

looking for vocab in ./datasets/AG_NEWS/vocab.torch
Load Prepared Data


In [3]:
embed_dim = 128
model = SentenceCNN(n_classes=n_classes, embed_dim=embed_dim, vocab_size=vocab_size)
model.load_state_dict(torch.load('./results/001/model_1'))
loss_fun = torch.nn.CrossEntropyLoss()

Code bei Sebastian

In [4]:
def _get_outputs(inference_fn, data, model, device, batch_size=256):

    _data = DataLoader(TensorDataset(data), shuffle=False, batch_size=batch_size)

    try:
        _y_out = []
        for x in _data:
            _y = inference_fn(x[0].to(device))
            _y_out.append(_y.cpu())
        return torch.vstack(_y_out)
    except RuntimeError as re:
        if "CUDA out of memory" in str(re):
            model.to('cpu')
            outputs = _get_outputs(inference_fn, data, model, 'cpu')
            model.to('cuda')
            return outputs
        else:
            raise re


def _get_predictions(inference_fn, data, model, device):
    return torch.argmax(_get_outputs(inference_fn, data, model, device), dim=1)


def validate(inference_fn, model, X, Y):

    if inference_fn is None:
        inference_fn = model

    model.eval()
    device = next(model.parameters()).device

    _y_pred = _get_predictions(inference_fn, X, model, device)
    model.train()

    acc = torch.mean((Y == _y_pred).to(torch.float)).detach().cpu().item()  # mean expects float, not bool (or int)
    return acc


# Aufrufsequence:
#
# acc_val.append(validate(inference_fn, model, *te_data))
#
# def train(model, optim, loss_fn, tr_data: DataLoader, te_data: tuple, inference_fn=None, \
#              n_batches_max=10, device='cuda'):
#
# active_model, last_acc = train(model, optimizer, loss_fun, tr_data, te_data,
#              inference_fn=model.forward_softmax, device=device, n_batches_max=n_batches)
#
# def imp_loop(path, model, init_model, best_model, device, optimizer, loss_fun, n_batches, n_epochs,
#            pruning_type, pruning_perc, n_iterations, tr_data: DataLoader, te_data: tuple):
#
# imp_loop(path, model, init_model, best_model, device, optimizer, loss_fun, n_batches, n_epochs,
#            pruning_type, pruning_perc, n_iterations, train_set, (X_test, Y_test))
#
# X_test, Y_test = next(iter(test_set))

In [5]:
# Ausgabe des Modells auf Sebastian Code mit test_set:

inference_fn = model.forward_softmax
acc_exp01 = validate(inference_fn, model, X_test, Y_test)

print(acc_exp01)

0.8793774247169495


In [6]:
# Untersuchungen mit angepassten Batch_Sizes in _get_outputs:

def _get_predictions_with_batch_s(inference_fn, data, model, device, batch_size):
    return torch.argmax(_get_outputs(inference_fn, data, model, device, batch_size), dim=1)


def validate_with_batch_s(inference_fn, model, X, Y, batch_size):

    if inference_fn is None:
        inference_fn = model

    model.eval()
    device = next(model.parameters()).device

    _y_pred = _get_predictions_with_batch_s(inference_fn, X, model, device, batch_size)
    model.train()

    acc = torch.mean((Y == _y_pred).to(torch.float)).detach().cpu().item()  # mean expects float, not bool (or int)
    return acc

In [7]:
batch_size = 1
acc_exp02 = validate_with_batch_s(inference_fn, model, X_test, Y_test, batch_size)

print(f'batch size = {batch_size}:')
print(acc_exp02)

batch_size = 1028
acc_exp03 = validate_with_batch_s(inference_fn, model, X_test, Y_test, batch_size)

print(f'batch size = {batch_size}:')
print(acc_exp03)

batch size = 1:
0.8793774247169495
batch size = 1028:
0.8793774247169495


In [8]:
# Anpassung mit inference_fn = forward_embedded_softmax
def _get_outputs_from_emb(inference_fn, data, model, device, batch_size=256):

    _data = DataLoader(TensorDataset(data), shuffle=False, batch_size=batch_size)

    try:
        _y_out = []
        for x in _data:
            _y = inference_fn(model.embed_input(x[0]).to(device))
            _y_out.append(_y.cpu())
        return torch.vstack(_y_out)
    except RuntimeError as re:
        if "CUDA out of memory" in str(re):
            model.to('cpu')
            outputs = _get_outputs(inference_fn, data, model, 'cpu')
            model.to('cuda')
            return outputs
        else:
            raise re


def _get_predictions_from_emb(inference_fn, data, model, device, batch_size=256):
    return torch.argmax(_get_outputs_from_emb(inference_fn, data, model, device, batch_size), dim=1)


def validate_from_emb(inference_fn, model, X, Y, batch_size=256):

    if inference_fn is None:
        inference_fn = model

    model.eval()
    device = next(model.parameters()).device

    _y_pred = _get_predictions_from_emb(inference_fn, X, model, device, batch_size)
    model.train()

    acc = torch.mean((Y == _y_pred).to(torch.float)).detach().cpu().item()  # mean expects float, not bool (or int)
    return acc

In [9]:
inference_fn = model.forward_embedded_softmax

print(f'batch size = Standart 256:')
acc_exp04 = validate_from_emb(inference_fn, model, X_test, Y_test)
print(acc_exp04)

batch_size = 1
acc_exp05 = validate_from_emb(inference_fn, model, X_test, Y_test, batch_size)

print(f'batch size = {batch_size}:')
print(acc_exp05)

batch_size = 1028
acc_exp06 = validate_from_emb(inference_fn, model, X_test, Y_test, batch_size)

print(f'batch size = {batch_size}:')
print(acc_exp06)

batch size = Standart 256:
0.8793774247169495
batch size = 1:
0.8793774247169495
batch size = 1028:
0.8793774247169495


In [10]:
def _get_outputs_emb_simple(inference_fn, data, model, device, batch_size=1):

    _data = DataLoader(TensorDataset(data), shuffle=False, batch_size=batch_size)

    _y_out = []
    for x in _data:
        _y = inference_fn(model.embed_input(x[0]).to(device))
        _y_out.append(_y.cpu())
    return torch.vstack(_y_out)

def _get_predictions_emb_simple(inference_fn, data, model, device):
    return torch.argmax(_get_outputs_emb_simple(inference_fn, data, model, device), dim=1)


def validate_emb_simple(inference_fn, model, X, Y):

    if inference_fn is None:
        inference_fn = model

    model.eval()
    device = next(model.parameters()).device

    _y_pred = _get_predictions_emb_simple(inference_fn, X, model, device)
    model.train()

    acc = torch.mean((Y == _y_pred).to(torch.float)).detach().cpu().item()  # mean expects float, not bool (or int)
    return acc

In [11]:
acc_exp07 = validate_emb_simple(inference_fn, model, X_test, Y_test)

print(acc_exp07)

0.8793774247169495


In [12]:
def _get_outputs_and_gradients(inference_fn, X, Y, model, loss_fun, device, batch_size=1):

    _data = DataLoader(TensorDataset(X), shuffle=False, batch_size=batch_size)
    _labels = DataLoader(TensorDataset(Y), shuffle=False, batch_size=batch_size)

    _y_out = []
    _grads = []
    for text, label in zip(_data, _labels):
        emb = model.embed_input(text[0]).to(device)
        emb.requires_grad = True
        _y = inference_fn(emb)
        _y_out.append(_y.cpu())
        _loss = loss_fun(_y, label[0])
        grad = torch.autograd.grad(_loss, emb, retain_graph=True)[0].data
        _grads.append(grad)
    return torch.vstack(_y_out), torch.vstack(_grads)

In [13]:
model.eval()
_y_outs, _grads = _get_outputs_and_gradients(inference_fn, X_test, Y_test, model, loss_fun, device)
_y_pred = torch.argmax(_y_outs, dim=1)
model.train()
print(torch.mean((Y_test == _y_pred).to(torch.float)).detach().cpu().item())

0.8793774247169495


In [14]:
correct = 0
for i in range(len(_y_pred)):
    if _y_pred[i] == Y_test[i]:
        correct += 1
print(correct/len(_y_pred))

0.8793774319066148


In [15]:
from rules import _get_gradients_and_outputs

_y_outs, _grads, _y_pred, acc = _get_gradients_and_outputs(inference_fn, X_test, Y_test, model, loss_fun, device)
print(acc)

0.8793774247169495
